# rETH vs S&P500
This example analyzes the trading behavior of a random Ethereum wallet that traded rETH (Rocketpool staked Ether) tokens and creates a data pipeline that continuously compares the market value of the tokens held to the theoretical performance of the same investment made into S&P500 (SPY) index.

This example is not at all meant to be an investing advise or any kind of commentary on cryptocurrency. It is meant to showcase the integration of on- and off-chain data in a single near real-time data processing pipeline. 

In [ ]:
%load_ext kamu

In [ ]:
import kamu

con = kamu.connect()

In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from datetime import datetime

pd.set_option('max_colwidth', None)

In [ ]:
%%sql -o reth_pool
select 
    event_time, 
    case 
        when event_name = 'TokensMinted' then 'Mint'
        when event_name = 'TokensBurned' then 'Burn'
    end as event_name, 
    avg(eth_amount / amount) as ratio 
from 'net.rocketpool.reth.mint-burn'
group by event_time, event_name
order by 1

In [ ]:
reth_pool.hvplot.step(
    x="event_time",
    by="event_name",
    width=900, height=600,
    legend='top_left', grid=True,
    title="ETH : rETH Ratio (Minting and Burning)",
)

In [ ]:
%%sql -o eth2usd
select event_time, open, close from 'com.cryptocompare.ohlcv.eth-usd' order by event_time

In [ ]:
eth2usd.hvplot.step(x="event_time", y="close", height=500, width=800)

In [ ]:
%%sql -o portfolio
select * from 'account.tokens.portfolio' order by block_time

In [ ]:
portfolio[
    portfolio.token_symbol == "rETH"
].hvplot.scatter(
    x="block_time",
    y="token_amount",
    color="orange",
    title="rETH Buy/Sell Transactions",
)

In [ ]:
%%sql -o reth_mv
select * from 'account.tokens.portfolio.market-value' order by event_time

In [ ]:
reth_mv.hvplot.line(
    x="event_time", 
    y=["token_book_value_eth", "token_market_value_eth"], 
    legend="bottom_right",
    title="rETH: Book vs Market Value in ETH",
    height=500, 
    width=800,
)

In [ ]:
reth_mv.hvplot.line(
    x="event_time", 
    y=["token_book_value_eth_as_usd", "token_market_value_usd"], 
    legend="bottom_right",
    title="rETH: Book vs Market Value in USD",
    height=500, 
    width=800,
)

In [ ]:
%%sql -o spy_ticks
select * from 'co.alphavantage.tickers.daily.spy'

In [ ]:
spy_ticks.hvplot.step(
    x="event_time", 
    y=["close"],
    title="SPY: Daily Close Price in USD",
    height=500, 
    width=800,
)

In [ ]:
%%sql -o market_value -q
select * from 'account.tokens.portfolio.market-value'

In [ ]:
%%sql -o alternative_market_value -q
select * from 'account.whatif.reth-vs-snp500.market-value'

In [ ]:
max_height = max(
    alternative_market_value["alt_spy_market_value_usd"].max(),
    market_value["token_market_value_usd"].max(),
    market_value["token_book_value_eth_as_usd"].max(),
)

alternative_market_value.hvplot.line(
    x="event_time", 
    y=["alt_spy_market_value_usd"],
    xlabel="Time",
    ylabel="USD",
    label="alt_spy_market_value_usd",
    title="rETH vs S&P500",
    height=500, 
    width=800,
) * market_value.hvplot.line(
    x="event_time", 
    y=["token_market_value_usd", "token_book_value_eth_as_usd"],
    legend="bottom_right",
)
# * hv.Spikes(
#     portfolio["block_time"]
# ).opts(
#     line_dash="dashed", 
#     color="green", 
#     line_width=.2,
#     line_alpha=0.7, 
#     spike_length=max_height
# )

# Batch alternative
Code below roughly shows how elements of this pipeline would look like in batch SQL.

```sql
%%sql -o reth_tx
with 

tx_fpv as (
    select
        *,
        cast(value as float) / power(10.0, 18) as value_fp
    from `account.transactions`
),

tx_deltas as (
    select
        *,
        case
            when to = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then value_fp
            when from = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then -value_fp
            else null
        end as eth_amount
    from tx_fpv
),


tokens_fpv as (
    select
        *,
        cast(value as float) / power(10.0, token_decimal) as value_fp
    from `account.tokens.transfers`
),

tokens_deltas as (
    select
        *,
        case
            when to = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then value_fp
            when from = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then -value_fp
            else null
        end as token_amount
    from tokens_fpv
),

tokens_eth as (
    select
        d.block_time,
        d.block_number,
        d.transaction_hash,
        d.token_symbol,
        d.token_amount,
        t.eth_amount
    from tokens_deltas as d
    left join tx_deltas as t
    on d.transaction_hash = t.transaction_hash
)

select
    block_time, 
    block_number,
    transaction_hash,
    token_amount,
    sum(token_amount) over (order by block_time) as token_balance,
    eth_amount,
    sum(eth_amount) over (order by block_time) as eth_balance
from tokens_eth
where token_symbol = "rETH"
order by 1
```